In [40]:
from sklearn import datasets
import random
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA 
import matplotlib.pyplot as plt
import pandas as pd

file_str = 'C:/Users/16010/Desktop/分数据集.xlsx'
data = pd.read_excel(file_str,header=None,sep=' ').values

m = data.shape[0]      #样本个数
n = data.shape[1] - 1    #特征个数

x_data = data[:,0:n]
y_data = data[:,n:n+1]

x_train = x_data[0:97]
x_test = x_data[97:124]
y_train = y_data[0:97].tolist()
y_test = y_data[97:124].tolist()

ss = StandardScaler()
zscore = ss.fit(x_train)
train_d = zscore.transform(x_train)
test_d = zscore.transform(x_test)

In [ ]:
print(len(y_test))
print(len(y_train))
print(y_test[1])

In [41]:
for i in range(len(y_train)):
    y_train[i] = y_train[i][0]
for i in range(len(y_test)):
    y_test[i] = y_test[i][0]

In [42]:
y_train_onehot = np.zeros([97,4])
y_test_onehot = np.zeros([26,4])
for i in range(len(y_test)):
    y_test_onehot[i][int(y_test[i])] = 1
for i in range(len(y_train)):
    y_train_onehot[i][int(y_train[i])] = 1

In [ ]:
y_train_onehot

In [43]:
import torch as pt

class MLP(pt.nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.fc1 = pt.nn.Linear(5,72)
        self.fc2 = pt.nn.Linear(72,36)
        self.fc3 = pt.nn.Linear(36,4)
        
    def forward(self,din):
        #din = din.view(-1,28*28)
        dout = pt.nn.functional.relu(self.fc1(din))
        dout = pt.nn.functional.relu(self.fc2(dout))
        return pt.nn.functional.softmax(self.fc3(dout))
model = MLP().cuda()
print(model)

MLP(
  (fc1): Linear(in_features=5, out_features=72, bias=True)
  (fc2): Linear(in_features=72, out_features=36, bias=True)
  (fc3): Linear(in_features=36, out_features=4, bias=True)
)


In [44]:
# loss func and optim
optimizer = pt.optim.SGD(model.parameters(),lr=0.01,momentum=0.9)
lossfunc = pt.nn.CrossEntropyLoss().cuda()

# accuarcy
def AccuarcyCompute(pred,label):
    pred = pred.cpu().data.numpy()
    label = label.cpu().data.numpy()
#     print(pred.shape(),label.shape())
    test_np = (np.argmax(pred,1) == label)
    test_np = np.float32(test_np)
    return np.mean(test_np)

In [59]:
for x in range(901):
    
    
    optimizer.zero_grad()

    inputs = pt.from_numpy(np.array(train_d)).float()
    labels = pt.from_numpy(np.array(y_train)).long()
    #print(labels)
    #print(inputs)
    inputs = pt.autograd.Variable(inputs).cuda()
    labels = pt.autograd.Variable(labels).cuda()

    outputs = model(inputs)

    loss = lossfunc(outputs,labels)
    loss.backward()

    optimizer.step()
    
    if x % 50 == 0:
        pred = outputs.cpu().data.numpy()
        label = labels.cpu().data.numpy()
        m = pred.argmax(1)
        acc = 0
        for i in range(len(pred)):
            if(m[i] == label[i]): acc += 1
            #print(m)
            #print(label)
        print(x,acc,'/',len(pred))
        print('==========')
            
        #print(x,":",AccuarcyCompute(outputs,labels))
    #print(outputs)
    
    #print(labels)

c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


0 60 / 97
50 62 / 97
100 63 / 97
150 64 / 97
200 64 / 97
250 64 / 97
300 64 / 97
350 64 / 97
400 64 / 97
450 64 / 97
500 64 / 97
550 64 / 97
600 64 / 97
650 66 / 97
700 65 / 97
750 67 / 97
800 67 / 97
850 69 / 97
900 69 / 97


In [60]:
accuarcy_list = []
inputs = pt.from_numpy(np.array(test_d)).float()
labels = pt.from_numpy(np.array(y_test)).long()
inputs = pt.autograd.Variable(inputs).cuda()
labels = pt.autograd.Variable(labels).cuda()
outputs = model(inputs)

pred = outputs.cpu().data.numpy()
label = labels.cpu().data.numpy()
m = pred.argmax(1)
acc = 0
for i in range(len(pred)):
    if(m[i] == label[i]): acc += 1
    #print(m)
    #print(label)
print(acc,'/',len(pred))
print('==========')

#accuarcy_list.append(AccuarcyCompute(outputs,labels))
#print(sum(accuarcy_list) / len(accuarcy_list))

900 15 / 26


c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
